In [ ]:
!pip install pymorphy2>None

In [ ]:
!pip install navec

In [ ]:
!pip install slovnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 990.9 kB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import pymorphy2
nltk.download('punkt')
nltk.download('stopwords')
from string import punctuation
punctuation = list(punctuation)
from nltk.stem.snowball import RussianStemmer
stemmer = RussianStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from gensim.models import Word2Vec
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Максем\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Максем\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import torch
from navec import Navec
from slovnet.model.emb import NavecEmbedding

In [ ]:
db_file_path = r'Голос/text_price.db'

# Создайте подключение к базе данных
conn = sqlite3.connect(db_file_path)

# Создайте курсор для выполнения SQL-запросов
cursor = conn.cursor()

In [ ]:
table_name = 'my_table'  # Замените на имя вашей таблицы

# Выполните SQL-запрос для получения списка колонок
query = f"PRAGMA table_info({table_name})"
cursor.execute(query)

# Извлеките результаты запроса
columns = [column[1] for column in cursor.fetchall()]

# Выведите список колонок
print(columns)

['id', 'region', 'text', 'price']


In [ ]:
df = pd.read_sql_query("SELECT * FROM my_table", conn)
conn.close()

In [ ]:
print(df)

        id region                                               text     price
0        1      1  Код объекта: 812760.\nПродается апартамент 15,...   1950000
1        2      1  Продаётся студия-апартаменты в строящемся доме...   3838584
2        3      1  Вашему вниманию предлагаются новые студии апар...   3701376
3        4      1  Продается студия в зеленом районе, в 400 метра...   3850000
4        5      1  Апартаменты в жилом доме (на плане БТИ выделен...   3900000
...    ...    ...                                                ...       ...
5491  5492   4885  Код объекта: 699633.\nПродаётся уютная 3-х ком...  29500000
5492  5493   4885  Код объекта: 699633.\nПродаётся уютная 3-х ком...  29500000
5493  5494   4885  Агентам не беспокоить! СВОБОДНАЯ продажа видов...  31010970
5494  5495   4885  Продается комфортная, двухуровневая очень тепл...  29900000
5495  5496   4885  Продается 3 -комнатная квартира площадью 187,2...  29996000

[5496 rows x 4 columns]


In [ ]:
print(df['region'].unique())

['1' '2' '4743' '4897' '4777' '4885']


In [ ]:
print('1', (df['region'] == '1').sum(), '\n', '2', (df['region'] == '2').sum(), '\n', '4743', (df['region'] == '4743').sum(), '\n', '4897', (df['region'] == '4897').sum(), '\n', '4777', (df['region'] == '4777').sum(), '\n', '4885', (df['region'] == '4885').sum())

1 1219 
 2 1593 
 4743 650 
 4897 1193 
 4777 448 
 4885 393


In [ ]:
# 'msc': '1', # москва
# 'spb': '2', # санкт-петербург
# 'ekb': '4743', # екатеринбург
# 'nsk': '4897', # новосибирск
# 'kzn': '4777', # казань
# 'nng': '4885', # нижний новгород

In [ ]:
msc_df = df[df['region'] == '1']
msc_df = msc_df.drop(columns='region')
msc_df.head()

,id,text,price
0,1,"Код объекта: 812760.\nПродается апартамент 15,...",1950000
1,2,Продаётся студия-апартаменты в строящемся доме...,3838584
2,3,Вашему вниманию предлагаются новые студии апар...,3701376
3,4,"Продается студия в зеленом районе, в 400 метра...",3850000
4,5,Апартаменты в жилом доме (на плане БТИ выделен...,3900000


In [ ]:
print(punctuation)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
!pip install razdel

^C


In [ ]:
from razdel import tokenize

In [ ]:
def normalize_text(s):
   #Очистка от html
    soup = BeautifulSoup(s, 'html.parser')
    soup.get_text()
   #Удаление скрипта
    for data in soup(['style', 'script']):
      data.decompose()
    script_out = ' '.join(soup.stripped_strings)
   # Токенизация
    tokens = [_.text for _ in list(tokenize(script_out))]
   # Удаляем пунктуацию
    tokens_without_punct = [i for i in tokens if i not in punctuation]
    low_tokens = [i.lower() for i in tokens_without_punct]
   # удаляем стоп-слова из нашего текста
    stopwords = nltk.corpus.stopwords.words('russian')
    words_without_stop = [i for i in low_tokens if i not in stopwords]
   # Лемматизация
    morph = pymorphy2.MorphAnalyzer()
    lemms = [morph.parse(i)[0].normal_form for i in words_without_stop]
   # Стемминг
    stems = [stemmer.stem(i) for i in lemms]
   # Вывод значения в строке
    total=''
    for el in stems:
      total+=el
      total+=' '
    return total

In [ ]:
msc_df['normalize'] = np.zeros(len(msc_df))
for i in range(len(msc_df)):
    msc_df.at[i, 'normalize'] = normalize_text(msc_df.at[i, 'text'])
msc_df.head()

C:\Users\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,id,text,price,normalize
0,1,"Код объекта: 812760.\nПродается апартамент 15,...",1950000,"код объект 812760 продава апартамент 15,1 м² ц..."
1,2,Продаётся студия-апартаменты в строящемся доме...,3838584,продава студия-апартамент стро дом дом 7 срок ...
2,3,Вашему вниманию предлагаются новые студии апар...,3701376,ваш вниман предлага нов студ апартамент качест...
3,4,"Продается студия в зеленом районе, в 400 метра...",3850000,продава студ зелен район 400 метр лесопарк кус...
4,5,Апартаменты в жилом доме (на плане БТИ выделен...,3900000,апартамент жил дом план бти выдел син цвет соб...


Navec

https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Скачать словарь по ссылке выше

In [ ]:
path = 'C:\\Users\\Максем\\Downloads\\navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
def get_embedding(text):
    words = text.split()
    ids = [navec.vocab[word] if word in navec.vocab else navec.vocab['<unk>'] for word in words]
    input_tensor = torch.tensor(ids)
    emb = NavecEmbedding(navec)
    embedding_tensor = emb(input_tensor)
    return embedding_tensor.tolist()

In [ ]:
msc_df['emb'] = msc_df['normalize'].apply(get_embedding)

C:\Users\anaconda3\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  torch.from_numpy(navec.pq.indexes),


In [ ]:
msc_df.head()

,id,text,price,normalize,emb
0,1,"Код объекта: 812760.\nПродается апартамент 15,...",1950000,"код объект 812760 продава апартамент 15,1 м² ц...","[[0.28818395733833313, 0.06682836264371872, -0..."
1,2,Продаётся студия-апартаменты в строящемся доме...,3838584,продава студия-апартамент стро дом дом 7 срок ...,"[[0.03691257908940315, 0.09328188747167587, 0...."
2,3,Вашему вниманию предлагаются новые студии апар...,3701376,ваш вниман предлага нов студ апартамент качест...,"[[-0.31351280212402344, -0.27878454327583313, ..."
3,4,"Продается студия в зеленом районе, в 400 метра...",3850000,продава студ зелен район 400 метр лесопарк кус...,"[[0.03691257908940315, 0.09328188747167587, 0...."
4,5,Апартаменты в жилом доме (на плане БТИ выделен...,3900000,апартамент жил дом план бти выдел син цвет соб...,"[[0.03691257908940315, 0.09328188747167587, 0...."


In [ ]:
print(msc_df['emb'][0])

[[0.28818395733833313, 0.06682836264371872, -0.555029571056366, -0.5257329940795898, -0.46980899572372437, 0.4997864067554474, 0.27070483565330505, -0.428549587726593, 0.4931786358356476, -0.17297056317329407, 0.36927035450935364, 0.520755410194397, -0.8356989622116089, -0.05341203138232231, -0.4475979208946228, -0.17291224002838135, -0.3477937579154968, -0.774146318435669, 0.27873727679252625, 0.37070831656455994, -0.01896664686501026, 0.42192620038986206, 0.2613401412963867, 0.15301038324832916, 0.5108431577682495, 0.386968195438385, 0.271733820438385, -0.14056457579135895, -0.15038006007671356, 0.5472779870033264, -0.4245743751525879, -0.5936159491539001, 0.13120727241039276, -0.031917065382003784, -0.4424634575843811, 0.0732751190662384, 0.441944420337677, 0.05715731903910637, -0.32185694575309753, -0.10974890738725662, -0.29705941677093506, 0.03234683722257614, -0.3123660385608673, -0.08413463085889816, 0.4609445333480835, 0.24418196082115173, -0.9863971471786499, 0.45322957634925

In [ ]:
print(len(msc_df['emb'][6]))
print(len(msc_df['emb'][5][2]))

155
300


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


In [ ]:
msc_df['emb_mean'] = msc_df['emb'].apply(lambda x: np.mean(x, axis=0))
msc_df['emb_std'] = msc_df['emb'].apply(lambda x: np.std(x, axis=0))
msc_df['emb_max'] = msc_df['emb'].apply(lambda x: np.max(x, axis=0))
msc_df['emb_min'] = msc_df['emb'].apply(lambda x: np.min(x, axis=0))
# Разделение данных на обучающий и тестовый наборы
X = np.vstack(msc_df['emb_mean'].to_numpy())
y = msc_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели CatBoostRegressor
model = CatBoostRegressor(iterations=5000, depth=6, learning_rate=0.3, loss_function='MAE', random_state=42)
model.fit(X_train, y_train)

# Предсказание цен для тестового набора
y_pred = model.predict(X_test)

# Вычисление метрик MAE и MAPE
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Преобразуем MAPE в проценты

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Выходные данные были обрезаны до нескольких последних строк (5000).
2:	learn: 5856712.4784087	total: 4.11ms	remaining: 6.84s
3:	learn: 5810576.9470183	total: 5.37ms	remaining: 6.7s
4:	learn: 5799977.4262549	total: 6.57ms	remaining: 6.56s
5:	learn: 5788449.8357198	total: 7.84ms	remaining: 6.53s
6:	learn: 5781963.8799841	total: 9.18ms	remaining: 6.54s
7:	learn: 5768137.0048982	total: 10.6ms	remaining: 6.61s
8:	learn: 5756448.2943267	total: 11.8ms	remaining: 6.52s
9:	learn: 5751129.3604587	total: 12.8ms	remaining: 6.38s
10:	learn: 5727795.0406265	total: 14ms	remaining: 6.33s
11:	learn: 5712007.3991921	total: 15.3ms	remaining: 6.34s
12:	learn: 5707468.4891958	total: 16.5ms	remaining: 6.34s
13:	learn: 5686013.3977745	total: 17.8ms	remaining: 6.33s
14:	learn: 5683899.8427025	total: 19.1ms	remaining: 6.34s
15:	learn: 5677072.8529231	total: 20.6ms	remaining: 6.43s
16:	learn: 5664072.8370923	total: 21.9ms	remaining: 6.42s
17:	learn: 5658935.5965861	total: 23.4ms	remaining: 6.47s
18:	learn: 5647